In [13]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import yaml

import plotly.graph_objects as go


In [17]:
with open('report.yml') as f:
    data = yaml.load(f.read())

/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [18]:
plt.figure()
for name, d in data.items():
    plt.plot(d['score_cdf_x'], d['score_cdf_y'], label=name)
plt.title('Final Score CDF')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:

def bins(n):
    bins = []
    for i in range(n):
        bins.append(2**(i+1))
    return bins

bar_values = { k: d['largest_hist'] for k, d in data.items() }
max_size = max([len(x) for x in bar_values.values()])

# Pad any short arrays with zeros, and normalize
for k in bar_values.keys():
    missing = max_size - len(bar_values[k])
    if missing > 0:
        bar_values[k] += [0] * missing
    
    bar_values[k] = np.array(bar_values[k], dtype=np.float)
    bar_values[k] *= 100 / bar_values[k].sum()
    
print(bar_values)

labels = ["g"+str(x) for x in bins(max_size)]

print(labels)

fig = go.FigureWidget(data=[
    go.Bar(name=name, x=labels, y=values) 
    for name, values in bar_values.items()
    ])
# Change the bar mode
fig.update_layout(barmode='group')
#fig.show()
# for name, d in data.items():
#     plt.hist(d['largest_hist'], bins(len(d['largest_hist'])), label=name)
    
# plt.legend()

{'max_free_space': array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.4       ,
        5.1       , 34.26666667, 47.2       , 12.83333333,  0.2       ]), 'max_free_space_3dir': array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.26666667,
        5.96666667, 33.3       , 44.83333333, 15.43333333,  0.2       ]), 'random': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.00000000e-01,
       8.30000000e+00, 3.95333333e+01, 4.49000000e+01, 6.83333333e+00,
       3.33333333e-02, 0.00000000e+00]), 'random_3dir': array([3.33333333e-02, 6.66666667e-02, 3.00000000e-01, 1.20000000e+00,
       8.86666667e+00, 3.29333333e+01, 4.07000000e+01, 1.54333333e+01,
       4.66666667e-01, 0.00000000e+00])}
['g2', 'g4', 'g8', 'g16', 'g32', 'g64', 'g128', 'g256', 'g512', 'g1024']


FigureWidget({
    'data': [{'name': 'max_free_space',
              'type': 'bar',
              'uid': '2363…